# SAR Amplitude Processing

This notebook demonstrates SAR (Synthetic Aperture Radar) processing.

## Contents
1. SAR Amplitude Data
2. Calibration and Sigma0
3. Speckle Filtering
4. dB Conversion

Note: This is a demonstration using synthetic data.

In [ ]:
import numpy as np
from unbihexium.sar.amplitude import (
    amplitude_to_db,
    calibrate_amplitude,
    compute_sigma0,
    speckle_filter,
)

## 1. Create Synthetic SAR Amplitude Data

In [ ]:
np.random.seed(42)

# SAR amplitude follows Rayleigh distribution
# Simulate complex SAR data and take amplitude
real = np.random.randn(256, 256)
imag = np.random.randn(256, 256)
amplitude = np.sqrt(real**2 + imag**2).astype(np.float32)

print(f"Amplitude shape: {amplitude.shape}")
print(f"Amplitude range: [{amplitude.min():.3f}, {amplitude.max():.3f}]")
print(f"Amplitude mean: {amplitude.mean():.3f}")

## 2. Calibration and Sigma0

In [ ]:
# Apply calibration
calibrated = calibrate_amplitude(amplitude, calibration_factor=1.0)
print(f"Calibrated mean: {calibrated.mean():.3f}")

# Compute sigma0 (normalized radar cross-section)
incidence_angle = 35.0  # degrees
sigma0 = compute_sigma0(calibrated, incidence_angle=np.radians(incidence_angle))
print(f"Sigma0 range: [{sigma0.min():.3f}, {sigma0.max():.3f}]")

## 3. Speckle Filtering

In [ ]:
# Apply speckle filter (Lee filter)
filtered = speckle_filter(sigma0, window_size=5)

print(f"Original std: {sigma0.std():.3f}")
print(f"Filtered std: {filtered.std():.3f}")
print(f"Noise reduction: {(1 - filtered.std() / sigma0.std()) * 100:.1f}%")

## 4. Convert to dB

In [ ]:
# Convert to decibels
sigma0_db = amplitude_to_db(filtered, floor=-30.0)

print(f"Sigma0 dB range: [{sigma0_db.min():.1f}, {sigma0_db.max():.1f}] dB")
print(f"Sigma0 dB mean: {sigma0_db.mean():.1f} dB")

## Processing Chain Summary

In [ ]:
# Full processing chain
def process_sar_amplitude(raw_amplitude, incidence_angle_deg=35.0):
    """Complete SAR amplitude processing chain."""
    # 1. Calibrate
    calibrated = calibrate_amplitude(raw_amplitude, calibration_factor=1.0)

    # 2. Compute sigma0
    sigma0 = compute_sigma0(calibrated, incidence_angle=np.radians(incidence_angle_deg))

    # 3. Filter speckle
    filtered = speckle_filter(sigma0, window_size=5)

    # 4. Convert to dB
    sigma0_db = amplitude_to_db(filtered, floor=-30.0)

    return sigma0_db


result = process_sar_amplitude(amplitude)
print(f"Final result shape: {result.shape}")
print(f"All values finite: {np.all(np.isfinite(result))}")

## Summary

This notebook demonstrated:
- SAR amplitude data simulation
- Radiometric calibration
- Sigma0 computation
- Speckle filtering
- Decibel conversion